In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image as keras_image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import os
import joblib

In [ ]:
from IPython.display import display, clear_output, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    # Invalid device or cannot modify virtual devices once initialized.
    print("No GPU?")
    clear_output()

In [ ]:
print("Setting up pre-trained keras ResNet50 model")
model = ResNet50(weights='imagenet')
print("Model ready")
clear_output()

In [ ]:
import h5py

In [ ]:
import urllib.request
if not os.path.exists('val_preds.h5'):
    print("Downloading MICP calibration data (190MB) - be patient!")
    urllib.request.urlretrieve("https://cml.rhul.ac.uk/people/ptocca/ILSVRC2012-CP/val_preds.h5",
                               'val_preds.h5')
    clear_output()

In [ ]:
with h5py.File('val_preds.h5','r') as f:
    preds_cal = f['preds'][:]

In [ ]:
def pValues(calibrationAlphas,testAlphas,randomized=False):
    testAlphas = np.array(testAlphas)
    sortedCalAlphas = np.sort(calibrationAlphas)
    
    leftPositions = np.searchsorted(sortedCalAlphas,testAlphas)
    
    if randomized:
        rightPositions = np.searchsorted(sortedCalAlphas,testAlphas,side='right')
        ties  = rightPositions-leftPositions+1   # ties in cal set plus the test alpha itself
        randomizedTies = ties * np.random.uniform(size=len(ties))
        return  (len(calibrationAlphas) - rightPositions + randomizedTies)/(len(calibrationAlphas)+1)
    else:
        return  (len(calibrationAlphas) - leftPositions + 1)/(len(calibrationAlphas)+1)


In [ ]:
def rev_score(scores,label):
    return -scores[:,label]


def ratio_own_to_max(scores, label):
    mask = np.ones(scores.shape[1],dtype=np.bool)
    mask[label] = False

    return np.amax(scores, axis=1, where=mask, initial=0) / scores[:,label]

In [ ]:
def micp_pValues(scores_cal,scores_test,y_cal,ncm):
    """Compute p-values for a Mondrian Inductive Conformal Predictor
    scores_cal,scores_test: arrays of shape (objects,labels) of scores for 
                            calibration set and test set
    y_cal: array of shape (objects,) with the labels of the calibration set
    ncm: function of scores and label, computing the NCM"""
    
    micp_pValues = []

    for i in range(scores_test.shape[1]):
        ncm_cal = ncm(scores_cal[y_cal==i], i)
        ncm_test = ncm(scores_test, i)
        p_i = pValues(ncm_cal,ncm_test)
        
        micp_pValues.append(p_i)

    micp_pValues = np.array(micp_pValues)
    
    return micp_pValues

In [ ]:
# ilsrvc_dir = "/mnt/d/Research/ILSVRC2012/"
ilsrvc_dir = "."

In [ ]:
gt_cal_file = os.path.join(ilsrvc_dir,"cal_gt.txt")
gt_test_file = os.path.join(ilsrvc_dir,"test_gt.txt")
lbls_file = os.path.join(ilsrvc_dir,"labels.txt")

In [ ]:
n_to_ki = {}
ki_to_synset = {}
with open(os.path.join(ilsrvc_dir,'synset_words.txt')) as f:
    for i,l in enumerate(f):
        ki_to_synset[i]=l[10:].strip()

In [ ]:
ground_truth_ki_cal = np.loadtxt(gt_cal_file,dtype=np.int)
ground_truth_ki_test = np.loadtxt(gt_test_file,dtype=np.int)

In [ ]:
import io

In [ ]:
import PIL.Image
import joblib

In [ ]:
mem = joblib.Memory('/dev/shm/joblib',verbose=0)

@mem.cache
def getImage(url):
    img_data = PIL.Image.open(urllib.request.urlopen(url))
    if img_data.mode != 'RGB':
        img_data = img_data.convert('RGB')
    img_data = img_data.resize((224,224),resample=PIL.Image.NEAREST)
    return img_data    

In [ ]:
def get_prob_sets(preds, eps):
    preds_as = np.argsort(-preds,axis=1)    # indices of the labels in descending order by prob
    preds_cumul = np.cumsum(np.take_along_axis(preds,preds_as,axis=1),axis=1)

    # We get the smallest sets that exceed 1-eps cumulative probability
    set_masks = preds_cumul<1-eps
    set_masks[:,1:] = set_masks[:,:-1]
    set_masks[:,0] = True

    sets = [(pr_as[m],pr[pr_as[m]]) for pr_as, m,pr in zip(preds_as,set_masks,preds)]
    return sets

In [ ]:
############################################################################################################
############################################################################################################

import pandas as pd

import panel as pn
import holoviews as hv
from bokeh.plotting import figure
from bokeh.models import LinearAxis, Range1d, ColumnDataSource

#hv.extension('bokeh')
pn.extension('mathjax')

In [ ]:
with open("ILSRVC_CP_Notes.html") as f:
    notes = f.read()

In [ ]:
initial_pic = 1000
initial_eps = 0.2
initial_ncm = "NegProb"

heading = pn.pane.HTML("<h1>Demo of CP using ResNet50 on ImageNet data</h1>", align="center")

desc = pn.pane.HTML("Basiliscus Horribilis")
desc_frame = pn.Row(pn.pane.HTML("ImageNet label:"), desc, align="center")

placeholder_img = np.zeros((224,224,3))
img = hv.RGB(data=placeholder_img).opts(height=400, width=400,
                                 xaxis=None, yaxis=None,
                                 toolbar=None, align="center")


In [ ]:
resnet50 = pn.widgets.DataFrame(disabled=True, name="ResNet50 Probability (top 5)")

CP = pn.widgets.DataFrame(disabled=True, name="Conformal Predictor at significance level 'eps'")


NCM = pn.widgets.RadioBoxGroup(name="NCM Choices",options=['NegProb','Ratio'],
                       value=initial_ncm)

labels = pn.Row(resnet50,NCM,CP)

In [ ]:
ncm_source = ColumnDataSource(data = dict(x=[0.0, 1.0], y=[0.0, 1.0]))

p_value_source = ColumnDataSource(data = dict(x=[0.5], y=[0.5]))


NCM_hist_output = figure(plot_height=400, plot_width=400,
                         title="Distribution of NCM",
                         y_range=[0.0, 1.0],
                         toolbar_location=None)
NCM_hist_output.xaxis.axis_label = "NonConformity Measure"
NCM_hist_output.yaxis.axis_label = "ECDF of NCM of calibration examples"


# Setting the second y axis range name and range
NCM_hist_output.extra_y_ranges = {"p-values": Range1d(start=1, end=0)}

# Adding the second axis to the plot.
NCM_hist_output.add_layout(LinearAxis(y_range_name="p-values",axis_label="p-value"), 'right')



NCM_hist_output.line(x="x", y="y", name='NCM',source=ncm_source)
NCM_hist_output.circle(x="x", y="y",source=p_value_source, color='red',
                                  y_range_name="p-values")


NCM_frame = pn.Column(pn.pane.HTML(""),  # Just to align nicely
                      NCM,
                      NCM_hist_output)
               

labels = pn.Row(resnet50,NCM_frame,CP)


In [ ]:
pic_idx = pn.widgets.IntSlider(value=initial_pic, name="Image",
                               start=1, end=2000, step=1, value_throttled=1, align='center')
eps_slider = pn.widgets.FloatSlider(value=initial_eps, name="Epsilon",
                             start=1e-8, end=1.0, step=0.01,
                             value_throttled=1,
                             align='center')

gui_tab = pn.Column(heading, img, desc_frame, pic_idx, eps_slider, labels)

help_tab = pn.pane.LaTeX(notes)

gui = pn.Tabs(('Demo', gui_tab), ('Notes',help_tab))


In [ ]:
from statsmodels.distributions.empirical_distribution import ECDF

class gui_data(object):
    pass


def update_pic(event):
    global pic_idx              # Debatable

    idx = pic_idx.value
      
    if 0:   # for development environment
        img_file = os.path.join(".","img","ILSVRC2012_valsub_%08d.JPEG"%i)
        img_data = keras_image.load_img(img_file, target_size=(224, 224))
    else:
        url="""https://cml.rhul.ac.uk/people/ptocca/ILSVRC2012-CP/img/ILSVRC2012_valsub_%08d.JPEG"""%idx
        img_data = getImage(url)
        
    output = io.BytesIO()
    img_data.save(output,format="PNG")

    # compute ResNet50 preds
    x = keras_image.img_to_array(img_data)
    img.data = x.astype(np.uint8)
    
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    gui_data.test_preds = model.predict(x)
    
    ## update ground truth widget
    lbl = ki_to_synset[ground_truth_ki_test[idx-1]]
    desc.object = lbl
    update_ResNet(event)
    update_CP(event)
    print("Finished update_pic")


    
def update_CP(event):
    ncm = NCM.value
    print(ncm)
    # compute CP
    if ncm=='NegProb':
        gui_data.ncm_f = rev_score
    elif ncm=='Ratio':
        gui_data.ncm_f = ratio_own_to_max
        
    gui_data.p_vals = micp_pValues(preds_cal, gui_data.test_preds, ground_truth_ki_cal, ncm=gui_data.ncm_f)

    eps = eps_slider.value
    gui_data.ps = np.argwhere(gui_data.p_vals>eps)[:,0].T
    ps_p_vals = gui_data.p_vals[gui_data.ps].flatten()
    gui_data.sorting_by_p_val = np.argsort(ps_p_vals)[::-1]
    df_dict = {}
    df_dict['p-value'] =  ["%0.3f"%p for p in ps_p_vals[gui_data.sorting_by_p_val]]
    df_dict['label'] = [ki_to_synset[k] for k in gui_data.ps[gui_data.sorting_by_p_val]]

    ## update CP widget
    CP.name = "CP (p-val) pred set at significance level %0.2f"%eps
    CP.data = pd.DataFrame(data=df_dict)

    update_NCM_histo(event)


def update_eps(event):
    update_ResNet(event)
    update_CP(event)
    
    
def update_ResNet(event):
    global resnet50
    eps = eps_slider.value
    
    resNet50_labels, resNet50_probs = get_prob_sets(gui_data.test_preds.reshape(1,-1), eps=eps)[0]

    df_dict = {}
    df_dict['Prob'] =  ["%0.3f"%p for p in resNet50_probs]
    df_dict['label'] = [ki_to_synset[k] for k in resNet50_labels]

    ## update resNet50 widget
    resnet50.name = "ResNet50 (prob) at aggr prob %0.2f"%(1-eps)
    resnet50.data = pd.DataFrame(data=df_dict)

    print(resnet50.data)

def update_NCM_histo(event):
    try:
        sel_p_val_label = gui_data.ps[gui_data.sorting_by_p_val[CP.selection[0]]]
    except IndexError:
        sel_p_val_label = np.argmax(gui_data.p_vals)
        
    ncm_cal = gui_data.ncm_f(preds_cal, sel_p_val_label)
    ncm_test = gui_data.ncm_f(gui_data.test_preds, sel_p_val_label)
    
    ncm_cal_mondrian = ncm_cal[ground_truth_ki_cal==sel_p_val_label]
    
    ecdf_ncm = ECDF(np.r_[ncm_cal_mondrian, ncm_test],side='left')   # TODO: check number of dimensions?

    label_synset = ki_to_synset[sel_p_val_label]
    if len(label_synset) > 15:
        label_synset = label_synset[:15] + "..."

    sel_p_val = gui_data.p_vals[sel_p_val_label]

    if ecdf_ncm is not None:
        ncm_source.data = dict(x = ecdf_ncm.x[1:], y=ecdf_ncm(ecdf_ncm.x[1:]))
        p_value_source.data = dict(x=[ncm_test[0]],y = [sel_p_val[0]])
        
    print(p_value_source.data)


In [ ]:
pic_idx.param.watch(update_pic,"value")
eps_slider.param.watch(update_eps, "value")
NCM.param.watch(update_CP, "value")
CP.param.watch(update_CP, "selection")

In [ ]:
# update_pic(None)

gui.servable()

In [ ]:
# srv.stop()

In [ ]:
i